In [1]:
import open3d as o3d
import numpy as np
import copy
import math

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Functions

In [2]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [3]:
def diceCoefficient(modelReduced,originalReduced):
    intersections = 0
    for i in range(len(modelReduced)):
        mX = modelReduced[i][0]
        mY = modelReduced[i][1]
        mZ = modelReduced[i][2]
        for j in range(len(originalReduced)):
            xDif = abs(originalReduced[j][0]-mX)
            yDif = abs(originalReduced[j][1]-mY)
            zDif = abs(originalReduced[j][2]-mZ)
            distance = math.sqrt(xDif*xDif+yDif*yDif+zDif*zDif)
            if distance<10:
                intersections += 1
                break
    return 2*intersections/(len(modelReduced)+len(originalReduced))

In [17]:
def hausdorffDistance(modelReduced,originalModel):
    hausdorff_distance = 0
    for i in range(len(modelReduced)):
        model_distance = math.sqrt(modelReduced[i][0]*modelReduced[i][0]+modelReduced[i][1]*modelReduced[i][1]+modelReduced[i][2]*modelReduced[i][2])
        for j in range(len(originalReduced)):
            original_distance = math.sqrt(originalReduced[j][0]*originalReduced[j][0]+originalReduced[j][1]*originalReduced[j][1]+originalReduced[j][2]*originalReduced[j][2])
            distance = abs(model_distance-original_distance)
            hausdorff_distance = distance if distance>hausdorff_distance else hausdorff_distance
    return hausdorff_distance

# Reconstruction evaluation

Se realiza el registro entre la nube de puntos del modelo obtenido y el molde original para realizar una evaluación del desempeño obtenido por la reconstrucción. Esto se realiza utilizando el algoritmo Iterative Closest Points. La transformación inicial del algoritmo es tal que rota los puntos del modelo -90° sobre el eje x y 180° sobre el eje y. De esta manera se tiene que
$$t_{ini} = \begin{bmatrix}
cos \theta & 0 & sen \theta \\
-sen \theta & 0 & cos \theta \\
0 & -1 & 0
\end{bmatrix}, \theta=\pi $$

In [4]:
modelMesh = o3d.io.read_point_cloud("test_images/009/carlos-rivas_mesh_poisson.ply")
originalMesh = o3d.io.read_point_cloud("test_images/009/carlos-rivas_mesh_oriented_original.ply")
# scale model mesh
modelMesh = modelMesh.scale(1000, center=modelMesh.get_center())
# compute normals of the point cloud
#down = modelMesh.voxel_down_sample(voxel_size=0.005)
#modelMesh.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.01,max_nn=40))
#originalMesh.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.01,max_nn=40))

In [5]:
# show original cloud points
o3d.visualization.draw_geometries([originalMesh,modelMesh])
#o3d.visualization.draw_geometries([originalMesh])

In [6]:
# set initial parameters
threshold = 10
trans_init = np.asarray([[-1, 0, 0, 0],
                         [0, 0, -1, 0],
                         [0, -1, 0, 0], [0.0, 0.0, 0.0, 1.0]])
draw_registration_result(modelMesh, originalMesh, trans_init) #initial alignment
evaluation = o3d.pipelines.registration.evaluate_registration(modelMesh, originalMesh,
                                                    threshold, trans_init)
print(evaluation)

RegistrationResult with fitness=6.597200e-01, inlier_rmse=5.317634e+00, and correspondence_set size of 6173
Access transformation to get result.


In [7]:
# Apply point-to-point ICP
reg_p2p = o3d.pipelines.registration.registration_icp(
    modelMesh, originalMesh, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
# homography transformation
print("Transformation is:")
print(reg_p2p.transformation)
# o3d.pipelines.registration.TransformationEstimationPointToPlane <- point-to-plane ICP

RegistrationResult with fitness=9.873891e-01, inlier_rmse=3.604513e+00, and correspondence_set size of 9239
Access transformation to get result.
Transformation is:
[[-0.97923214 -0.17707311  0.09873966 -1.17355615]
 [-0.14259583  0.25532672 -0.9562817  14.58906849]
 [ 0.14412091 -0.95050165 -0.27527402 -1.02160842]
 [ 0.          0.          0.          1.        ]]


In [8]:
# show first registration
draw_registration_result(modelMesh, originalMesh, reg_p2p.transformation)

In [9]:
# transform original image
modelMesh = modelMesh.transform(reg_p2p.transformation)

In [10]:
# show original cloud points
o3d.visualization.draw_geometries([originalMesh,modelMesh])

Luego, se utiliza el coeficiente de Dice para evaluar el desempeño del algoritmo utilizado para la reconstrucción del modelo. Este está definido como
$$ DSC = \frac{2|X \cap Y|}{|X|+|Y|} $$

In [11]:
# cloud points to array data
pcd_model = np.asarray(modelMesh.points)
pcd_original = np.asarray(originalMesh.points)
# reduce cloud sample
mainFactor = 10
reduceFactor = mainFactor*int(len(pcd_original)/len(pcd_model))
modelReduced = modelMesh.uniform_down_sample(mainFactor)
originalReduced = originalMesh.uniform_down_sample(reduceFactor)

In [12]:
o3d.visualization.draw_geometries([originalReduced,modelReduced])

In [13]:
# get point cloud as array
modelReduced = np.asarray(modelReduced.points)
originalReduced = np.asarray(originalReduced.points)

In [14]:
# compute dice coefficient
diceCoeff = diceCoefficient(modelReduced,originalReduced)
print(f'El coeficiente de dice es {diceCoeff}')

El coeficiente de dice es 0.8839884947267498


In [18]:
# compute Hausdorff similarity
hDistance = hausdorffDistance(modelReduced,originalReduced)
print(f'La distancia Hausdorff entre los dos conjuntos es {hDistance}')

La distancia Hausdorff entre los dos conjuntos es 72.34941339954165
